In [1]:
from utils.utils import get_root_project
from mlflow.models.signature import infer_signature
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import mlflow
import os

In [2]:
tracking_server = get_root_project() / 'MLFlow' / 'mlruns'

os.makedirs(tracking_server, exist_ok = True)
mlflow.set_tracking_uri(tracking_server.as_uri())

In [3]:
experiment_name = "dtree_model"
try: 
    mlflow.create_experiment(name = experiment_name)
except Exception as e:
    print(e)

mlflow.set_experiment(experiment_name = experiment_name)

<Experiment: artifact_location='file:///Users/sachintripathi/Documents/Py_files/EXL_MLOps/MLFlow/mlruns/869131075508411226', creation_time=1734081375763, experiment_id='869131075508411226', last_update_time=1734081375763, lifecycle_stage='active', name='dtree_model', tags={}>

In [4]:
x, y = make_classification(n_samples = 1000, n_features = 5, n_classes = 2, random_state = 42)

x_df = pd.DataFrame(x, columns = [f"feature_{i+1}" for i in range(x.shape[1])])
y_df = pd.DataFrame(y, columns = ['target'])
x_train, x_test, y_train, y_test = train_test_split(x_df, y_df, test_size = 0.2, random_state = 42)

model_signature = infer_signature(x_train, y_train)

/Users/sachintripathi/Documents/Py_files/dev/lib/python3.12/site-packages/mlflow/types/utils.py:435: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


In [10]:
baseline_model = DecisionTreeClassifier(random_state = 42)
baseline_model.fit(x_train, y_train)
predictions = baseline_model.predict(x_test)
# logging the baseline model
with mlflow.start_run(run_name = "baseline") as baseline_run: 
    mlflow.log_params(baseline_model.get_params())
    artifact_path = "baseline_model"
    mlflow.sklearn.log_model(baseline_model, artifact_path, signature = model_signature)
    eval_df = pd.DataFrame({
    "target": y_test.to_numpy().flatten(), 
    "predictions": predictions
    })
    result = mlflow.evaluate(
    data = eval_df, 
    model_type = "classifier", 
    targets = "target", 
    predictions = "predictions" 
    )


/Users/sachintripathi/Documents/Py_files/dev/lib/python3.12/site-packages/mlflow/types/utils.py:435: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2024/12/13 14:49:45 WARNING mlflow.models.evaluation.evaluators.classifier: According to the evaluation dataset label values, the model type looks like None, but you specified model type 'classifier'. Please verify